In [40]:
import tensorflow as tf
import keras
import os
import numpy as np 
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNet
from keras.models import Model,Sequential
from keras.layers import Dense,GlobalAveragePooling2D,Conv2D,Dropout,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from keras.preprocessing import image
from keras.models import load_model

In [23]:
# MobileNet is designed to work with images of dim 224,224
img_rows,img_cols = 224,224

MobileNet = MobileNet(weights='imagenet',include_top=False,input_shape=(img_rows,img_cols,3))

# Here we freeze the last 4 layers
# Layers are set to trainable as True by default

for layer in MobileNet.layers:
    layer.trainable = False
    
top_model = MobileNet.output
top_model = GlobalAveragePooling2D()(top_model)
top_model = Dense(1024,activation='relu')(top_model)
top_model = Dense(1024,activation='relu')(top_model)
top_model = Dense(512,activation='relu')(top_model)
top_model = Dense(num_classes,activation='softmax')(top_model)


num_classes = 10 

model = Model(inputs = MobileNet.input, outputs = top_model)

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [26]:
TRAIN_DIR=r'C:\Stud\Python\Projects\Monkey_Breed\training\training\\'
VALIDATION_DIR=r'C:\Stud\Python\Projects\Monkey_Breed\validation\validation\\'

In [28]:
train_datagen = ImageDataGenerator(rescale=1./255,
                   horizontal_flip=True,
                   shear_range=0.2,
                   rotation_range=40,
                   zoom_range=0.2,
                   width_shift_range=0.3,
                   height_shift_range=0.3,
                   fill_mode='nearest'
                  )

validation_datagen=ImageDataGenerator(rescale=1./255)

batch_size=32

train_generator=train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(img_size[0],img_size[1]),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator=validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(img_size[0],img_size[1]),
    batch_size=batch_size,
    class_mode='categorical')



Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [29]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,verbose=1,
                              patience=4, min_lr=0.0001)

checkpoint = ModelCheckpoint("monkey_brred_model.h5", monitor='val_loss',
                             save_best_only=True, mode=min, verbose=1)

earlystop = EarlyStopping(monitor='val_loss',min_delta=0,patience=10, verbose=1,restore_best_weights=True)
                             

callbacks = [ checkpoint, earlystop ,reduce_lr]

C:\Users\KARTHIK\Anaconda3\lib\site-packages\keras\callbacks.py:407: RuntimeWarning: ModelCheckpoint mode <built-in function min> is unknown, fallback to auto mode.
  RuntimeWarning)


In [30]:
print(train_generator.n)

1098


In [33]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
epochs = 25
history=model.fit_generator(train_generator,
                            steps_per_epoch = train_generator.n // batch_size,
                            epochs=epochs,
                            validation_data = validation_generator,
                            callbacks=callbacks,
                            validation_steps=validation_generator.n // batch_size)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/25
34/34 [==============================] - 163s 5s/step - loss: 1.5602 - acc: 0.4752 - val_loss: 0.3627 - val_acc: 0.8867

Epoch 00001: val_loss improved from inf to 0.36268, saving model to monkey_brred_model.h5
Epoch 2/25
34/34 [==============================] - 123s 4s/step - loss: 0.5683 - acc: 0.8179 - val_loss: 0.3955 - val_acc: 0.8792

Epoch 00002: val_loss did not improve from 0.36268
Epoch 3/25
34/34 [==============================] - 115s 3s/step - loss: 0.3699 - acc: 0.8696 - val_loss: 0.1429 - val_acc: 0.9542

Epoch 00003: val_loss improved from 0.36268 to 0.14288, saving model to monkey_brred_model.h5
Epoch 4/25
34/34 [==============================] - 102s 3s/step - loss: 0.3699 - acc: 0.8847 - val_loss: 0.1861 - val_acc: 0.9583

Epoch 00004: val_loss did not improve from 0.14288
Epoch 5/25
34/34 [==============================] - 103s 3s/step - loss: 0.4020 - acc: 0.868

In [34]:
model.save('mokey_model.h5')

# Testing Our Model

In [43]:

path=r'C:\Stud\Python\Projects\Monkey_Breed\Implementation\\'


model = load_model(path + 'monkey_bred_weights.h5')

img_rows,img_cols = 224,224

class_labels = [
	'mantled_howler', 
	'patas_monkey', 
	'bald_uakari', 
	'japanese_macaque', 
	'pygmy_marmoset', 
	'white_headed_capuchin',
	'silvery_marmoset',
	'common_squirrel_monkey', 
	'black_headed_night_monkey',
	'nilgiri_langur' 
	]


img = image.load_img(path+'test.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x.astype('float32')/255
pred = np.argmax(model.predict(x))
print("It's a {}.".format(class_labels[pred])) 


[[2.9192654e-08 1.0083945e-07 2.1159111e-07 3.4632027e-05 5.8693340e-04
  6.0423121e-05 9.9931097e-01 1.1763944e-07 4.3208401e-09 6.6880348e-06]]
It's a silvery_marmoset.
